In [1]:
!pip install kgtk

In [2]:
cd sample_data/

/content/sample_data


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd ../drive/MyDrive/Colab\ Notebooks/rxnorm_fda_kg/notebooks

/content/drive/MyDrive/Colab Notebooks/rxnorm_fda_kg/notebooks


In [5]:
import csv
import json
from SPARQLWrapper import SPARQLWrapper, JSON
import sys
import re

In [6]:
def write_tsv(file_name, data_lists):
	with open(file_name, "w", encoding="utf-8") as f:
		f.write("node1\tlabel\tnode2\n")
		for data in data_lists:
			line = "\t".join(data)
			line += "\n"
			f.write(line)

In [7]:
subject_list= ['1656341', '317541', '69749', '1656328', '1656339', '1656346', '1656351', '1656356', '1656342', '1656350', '1656355', '1656343', '1656340', '1656349', '1656354', '1656334', '1656335', '1656347', '1656348', '1656352', '1656353', '1656338', '1656336', '1656337', '1656344', '1656345', '1151131', '1151133']

In [8]:
triples_all=[]

In [9]:
rxcui_tty_dict={}
rxcui_identifier_qual_dict={}
with open('../data/rxnorm/rxnconso.csv', encoding="utf-8") as f:
    reader = csv.reader(f)
    
    header_list = next(reader)
    print("Header=", header_list)
    
    
    for row in reader:
        triples=[]
        
        if row[0] in subject_list and row[11] != 'RXNORM':
        #if row[11] != 'RXNORM':
            #print("Subject Identifier=",row[0])
            triples.append("\""+row[0]+"\"")
            triples.append(row[11])
            triples.append("\""+row[13]+"\"")
            rxcui_identifier_qual_dict_key=row[0]+"_"+row[11]
            if rxcui_identifier_qual_dict_key not in rxcui_identifier_qual_dict:
              rxcui_identifier_qual_dict[rxcui_identifier_qual_dict_key]=[]
              rxcui_identifier_qual_dict[rxcui_identifier_qual_dict_key].append(str(row[14]))
            else:
              rxcui_identifier_qual_dict[rxcui_identifier_qual_dict_key].append(str(row[14]))
            triples_all.append(triples)
                #print(row[0], row[11], row[13])
        triples_lang=[]
        triples_rxcui=[]
        triples_tty=[]
        triples_name=[]
        triples_suppress=[]
        triples_synonym=[]
        
        if row[0] in subject_list and row[11] == 'RXNORM' and row[12] not in ('SY', 'TMSY', 'PSN'):
        #if row[11] == 'RXNORM' and row[12] not in ('SY', 'TMSY', 'PSN'):
            #print("Subject Triples=",row[0])
            triples_lang.append("\""+row[0]+"\"")
            triples_lang.append("language")
            triples_lang.append("\""+row[1]+"\"")
            
            triples_rxcui.append("\""+row[0]+"\"")
            triples_rxcui.append("rxcui")
            triples_rxcui.append("\""+row[13]+"\"")
            
            triples_tty.append("\""+row[0]+"\"")
            triples_tty.append("tty")
            triples_tty.append("\""+row[12]+"\"")
            rxcui_tty_dict[row[0]]=row[12]
            
            triples_name.append("\""+row[0]+"\"")
            triples_name.append("name")
            triples_name.append("\""+row[14]+"\"")
            
            triples_suppress.append("\""+row[0]+"\"")
            triples_suppress.append("suppress")
            triples_suppress.append("\""+row[16]+"\"")
            
            if triples_lang != []:
                triples_all.append(triples_lang)
            if triples_rxcui != []:
                triples_all.append(triples_rxcui)
            if triples_tty != []:
                triples_all.append(triples_tty)
            if triples_name != []:
                triples_all.append(triples_name)
            if triples_suppress !=[]:
                triples_all.append(triples_suppress)
        
        if row[0] in subject_list and row[11] == 'RXNORM' and row[12] in ('SY', 'TMSY', 'PSN'):
        #if row[11] == 'RXNORM' and row[12] in ('SY', 'TMSY', 'PSN'):
            #print("Subject Triples=",row[0])
            triples_synonym.append("\""+row[0]+"\"")
            triples_synonym.append("synonym")
            triples_synonym.append("\""+row[14]+"\"")
            if triples_synonym !=[]:
                triples_all.append(triples_synonym)

print("Added Triples from RXNCONSO")

Header= ['RXCUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF', 'RXAUI', 'SAUI', 'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE', 'STR', 'SRL', 'SUPPRESS', 'CVF']
Added Triples from RXNCONSO


In [10]:
write_tsv('../results/rxnorm/intermediate_triples/rxnorm_triples.tsv', triples_all)

In [11]:
rxcui_rel_dict={}
with open('../data/rxnorm/rxnrel.csv', encoding="utf-8") as f:
    reader = csv.reader(f)
    
    header_list = next(reader)
    print("Header=", header_list)
    
    
    for row in reader:
        triples=[]
        
        if row[2]=='AUI':
            break
        x=row[4].split(".",1)[0]
    
        if x in subject_list and row[2]=='CUI' and row[6]=='CUI':
        #if row[2]=='CUI' and row[6]=='CUI':
            subject=x
            #print("Subject Rel=",x)
            triples.append("\""+subject+"\"")
            triples.append(row[7])
            obj=row[0].split(".",1)[0]
            triples.append("\""+obj+"\"")
            if subject not in rxcui_rel_dict:
                 rxcui_rel_dict[subject]=[]
                 rxcui_rel_dict[subject].append([row[7],obj])
            else:
                rxcui_rel_dict[subject].append([row[7],obj])
            #rxcui_rel_dict[subject].add(obj)
            triples_all.append(triples)
           # print(row[4], row[7], row[0])
print("Added Triples from RXNREL")

Header= ['RXCUI1', 'RXAUI1', 'STYPE1', 'REL', 'RXCUI2', 'RXAUI2', 'STYPE2', 'RELA', 'RUI', 'SRUI', 'SAB', 'SL', 'RG', 'DIR', 'SUPPRESS', 'CVF']
Added Triples from RXNREL


In [12]:
write_tsv('../results/rxnorm/intermediate_triples/rxnorm_triples.tsv', triples_all)

In [13]:
ndc11_dict={}
ndc2seg_dict={}
ndc3seg_dict={}
spl_setid_dict={}
application_dict={}

In [14]:
with open('../data/rxnorm/rxnsat.csv', encoding="utf-8") as f:
    reader = csv.reader(f)
    
    header_list = next(reader)
    print("Header=", header_list)
    
    
    for row in reader:
        
          
        if row[0] in subject_list and ((row[9]== 'RXNORM') or row[8] in ('NDC', 'SPL_SET_ID', 'NDA', 'ANDA')):
        #if row[9]== 'RXNORM' and row[8] in ('NDC', 'UMLSCUI'):
            #print("Subject SAT=",row[0])
            
            counter = row[10].count('-') 
            if row[8] =='NDC':
                if counter==0:
                    ndc11_dict[row[10]]=row[0]
                    triples=[]
                    triples.append("\""+row[0]+"\"")
                    triples.append(row[8])
                    triples.append("\""+row[10]+"\"")
                    triples_all.append(triples)
                    
                elif counter==1:
                    ndc2seg_dict[row[10]]=row[0]
                    triples=[]
                    triples.append("\""+row[0]+"\"")
                    triples.append("NDC 2 Segment")
                    triples.append("\""+row[10]+"\"")
                    triples_all.append(triples)
                elif counter==2:
                    ndc3seg_dict[row[10]]=row[0]
                    triples=[]
                    triples.append("\""+row[0]+"\"")
                    triples.append("NDC 3 Segment")
                    triples.append("\""+row[10]+"\"")
                    triples_all.append(triples)
                    
                    ndc2seg='-'.join(row[10].split('-')[0:2])
                    ndc2seg_dict[ndc2seg]=row[0]
                    triples=[]
                    triples.append("\""+row[0]+"\"")
                    triples.append("NDC 2 Segment")
                    triples.append("\""+ndc2seg+"\"")
                    triples_all.append(triples)
            elif row[8] in ('NDA', 'ANDA'):
                application_dict[row[10]]=row[0]
                triples=[]
                triples.append("\""+row[0]+"\"")
                triples.append("DrugsFDA Application Number")
                triples.append("\""+row[10]+"\"")
                triples_all.append(triples)
            elif row[8] in ('SPL_SET_ID'):
                
                spl_setid_dict[row[10]]=row[0]
                triples=[]
                triples.append("\""+row[0]+"\"")
                triples.append(row[8])
                triples.append("\""+row[10]+"\"")
                triples_all.append(triples)
            else:
                triples=[]
                triples.append("\""+row[0]+"\"")
                triples.append(row[8])
                triples.append("\""+row[10]+"\"")
                triples_all.append(triples)
print("Added Triples from RXNSAT")

Header= ['RXCUI', 'LUI', 'SUI', 'RXAUI', 'STYPE', 'CODE', 'ATUI', 'SATUI', 'ATN', 'SAB', 'ATV', 'SUPPRESS', 'CVF']
Added Triples from RXNSAT


In [15]:
write_tsv('../results/rxnorm/intermediate_triples/rxnorm_triples.tsv', triples_all)

In [16]:
identifier_source_list=[
"USP"
,"GS"
,"SNOMEDCT_US"
,"VANDF"
,"MTHSPL"
,"NDDF"
,"ATC"
,"MMSL"
,"MSH"
,"DRUGBANK"
,"MMX"
,"NDC"
,"NDC 3 Segment"
,"NDC 2 Segment"
,"SPL_SET_ID"
,"DrugsFDA Application Number"
,"spl_id"
,"product_id"
,"unii"
,"event_id"
,"upc"]

In [17]:
rela_types_list=[
'consists_of',
'constitutes',
'contained_in',
'contains',
'dose_form_of',
'form_of',
'has_dose_form',
'doseformgroup_of',
'has_form',
'has_ingredient',
'has_ingredients',
'has_part',
'has_precise_ingredient',
'has_quantified_form',
'has_tradename',
'has_doseformgroup',
'ingredient_of',
'ingredients_of',
'inverse_isa',
'isa',
'part_of',
'precise_ingredient_of',
'quantified_form_of',
'reformulated_to',
'reformulation_of',
'tradename_of'
]

In [18]:
tty_dict={
   'BN': 'brand_name'	    
  ,'BPCK': 'branded_pack'	    
  ,'DF': 'dose_form'	    
  ,'DFG': 'dose_form_group'	
  ,'ET': 'dose_form_entry_term'    
  ,'GPCK': 'generic_pack'	    
  ,'IN': 'ingredient'	    
  ,'MIN': 'multiple_ingredients'	    
  ,'PIN': 'precise_ingredient'	    
  ,'SBD': 'branded_drug'
  ,'SBDC': 'branded_drug_component'
  ,'SBDF': 'branded_dose_form'
  ,'SBDG': 'branded_dose_form_group'
  ,'SCD': 'clinical_drug'
  ,'SCDC': 'clinical_drug_component'
  ,'SCDF': 'clinical_dose_form'
  ,'SCDG': 'clinical_dose_form_group'}

In [19]:
pred_wikidata_dict={
        'instanceOf':'P31'
        ,'rxcui':'P3345'
        ,'UMLSCUI':'P2892'
        ,'SNOMEDCT_US':'P5806'
        ,'MSH':'P486'
        ,'DRUGBANK':'P715'
        ,'NDC': 'P3640'
        ,'ATC': 'P267'
        ,'unii':'P652'
        ,'manufacturer_name':'P176'
        ,'sponsor_name':'P859'
        ,'postal_code':'P281'
        }

In [20]:
pred_notinwikidata_dict_rxnorm={}

In [21]:
output_rows_qnode_pnode_rxnorm=[]
output_rows_qnode_prxnode_rxnorm=[]
output_rows_qrxnode_pnode_rxnorm=[]
output_rows_qrxnode_prxnode_rxnorm=[]
output_rows_prxnode_rxnorm=[]
output_rows_prxnode_edges_rxnorm=[]
output_rows_prxnode_datatype_rxnorm=[]

In [22]:
def get_query(identifier):
    query = """#All items with a property
    # Sample to query all values of a property
    # Property talk pages on Wikidata include basic queries adapted to each property
    SELECT
      ?item ?itemLabel
      ?value ?valueLabel
    # valueLabel is only useful for properties with item-datatype
    WHERE 
    {
      ?item wdt:"""+pred_wikidata_dict[identifier]+""" ?value
              
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    # remove or change limit for more results
    """
    return query

In [23]:
def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

In [24]:
qnode_dict_inwiki={}
endpoint_url = "https://query.wikidata.org/sparql"
identifier="rxcui"
query=get_query(identifier)

In [25]:
results = get_results(endpoint_url, query)
   
   
for result in results["results"]["bindings"]:
    
    if result['item']['type']=='uri':
        qnode=result['item']['value']
        identifier_value=result['value']['value']
        qnode=qnode.split("entity/",1)[1]
        if identifier=='rxcui':
            qnode_dict_inwiki[identifier_value]=qnode

In [26]:
triples_qnode_all=[]
for x in qnode_dict_inwiki:
    triples_qnode=[]
    triples_qnode.append(qnode_dict_inwiki[x])
    triples_qnode.append('P3345')
    triples_qnode.append("\""+x+"\"")
    triples_qnode_all.append(triples_qnode)

In [27]:
write_tsv("../results/rxnorm/kgtk_triples/nodes/qnode_pnode_rxnorm.tsv", triples_qnode_all)

In [28]:
with open('../results/rxnorm/intermediate_triples/rxnorm_triples.tsv', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter='\t')
    
    header_list = next(reader)
    print("Header=", header_list)
    
    
    qnode_dict_notinwiki={}
    label=''
    desc=''
    
    for row in reader:
        output_row=[]
        
       
        #output_row.append(str(row[0]))
        
        if row[1]=='name':
            label=str(row[2])
            
        if row[1]=='tty':
            desc=str(tty_dict[row[2]].replace("_"," "))
           
        if row[0] not in qnode_dict_notinwiki and row[0] not in qnode_dict_inwiki:
            qnode_dict_notinwiki[row[0]]='QRX'+str(row[0])
            #label_desc_dict[row[0]]=label+'-'+desc
        
#print(qnode_dict_notinwiki)

Header= ['node1', 'label', 'node2']


In [29]:
for x in qnode_dict_notinwiki.keys():
   
    #print(x)
    output_row=[]  
   
    output_row.append(qnode_dict_notinwiki[x])
    output_row.append(pred_wikidata_dict['instanceOf'])    
    output_row.append("Q28885102")
    #print(output_row)
    output_rows_qrxnode_pnode_rxnorm.append(output_row)

In [30]:
write_tsv("../results/rxnorm/kgtk_triples/nodes/qrxnode_pnode_rxnorm.tsv", output_rows_qrxnode_pnode_rxnorm)

In [31]:
active_ingredients_dict={}
with open('../data/fda/drug-ndc-0001-of-0001.json') as ndc_json:
    data=json.load(ndc_json)
    #Check meta-data
    #print(data['meta'])
    triples_product_ndc=[]
    triples_spl_ndc=[]
    triples_package_ndc=[]
    for entry in data['results']:
        
        product_ndc=entry['product_ndc']
        #print(product_ndc)
        klist=entry.keys()
        if product_ndc in ndc2seg_dict:
        
            
            #print(klist)
            
            #if product_ndc=='0002-8824':
            for k in klist:
                triple=[]
                #marketing_start_date= entry['marketing_start_date']
                if k not in ('product_ndc', 'openfda', 'packaging', 'route', 'pharm_class', 'active_ingredients'):
                    triple.append(product_ndc)
                    triple.append(str(k))
                    triple.append(str(entry[k]))
                    triples_product_ndc.append(triple)
                elif k in ('route', 'pharm_class'):
                    for field in entry[k]:
                        triple=[]
                        triple.append(product_ndc)
                        triple.append(str(k))
                        triple.append(str(field))
                        triples_product_ndc.append(triple)
                elif k in ('active_ingredients'):
                    for field in entry[k]:
                        #if 'strength' in field and 'name' in field:
                        triple=[]
                        triple.append(product_ndc)
                        triple.append(str(k))
                        triple.append(str(field['name']))
                        active_ingredients_dict_key=ndc2seg_dict[product_ndc]+"_"+str(field['name'])
                        active_ingredients_dict[active_ingredients_dict_key]=str(field['strength'])
                        triples_product_ndc.append(triple)
                    
         
      
        
        if 'spl_set_id' in entry['openfda'].keys():
            spl_set_id=str(entry['openfda']['spl_set_id']).replace("['","").replace("']","")
            #print(spl_set_id)
            
            #if product_ndc in ndc2seg_dict or spl_set_id in spl_setid_dict:
            if spl_set_id in spl_setid_dict:
                openfda=entry['openfda']
                slist=openfda.keys()
                
                """
                triple=[]
                triple.append(product_ndc)
                triple.append('spl_set_id')
                triple.append(str(spl_set_id))
                triples_product_ndc.append(triple)
                """
                
                for p in slist:
                    triple=[]
                    #marketing_start_date= entry['marketing_start_date']
                    if p != 'spl_set_id':
                        for field in openfda[p]:
                            triple=[]
                            triple.append(str(spl_set_id))
                            triple.append(str(p))
                            triple.append(str(field))
                            triples_spl_ndc.append(triple)
        
        if 'packaging' in klist:
            for package in entry['packaging']:
                package_ndc=package['package_ndc']
                #print(package_ndc)
                
                #if product_ndc in ndc2seg_dict or spl_set_id in spl_setid_dict or package_ndc in ndc3seg_dict:
                if package_ndc in ndc3seg_dict:
                    #print(package_ndc)
                    #packaging=entry['packaging']
                    plist=package.keys()
                    
                    """
                    triple=[]
                    triple.append(product_ndc)
                    triple.append('package_ndc')
                    triple.append(str(package_ndc))
                    triples_product_ndc.append(triple)
                    """
                    
                    for p in plist:
                        triple=[]
                        #marketing_start_date= entry['marketing_start_date']
                        if p != 'package_ndc':
                            triple.append(str(package_ndc))
                            triple.append(str(p))
                            triple.append(str(package[p]))
                            triples_package_ndc.append(triple)
print("Added triples from FDA NDC")

Added triples from FDA NDC


In [32]:
write_tsv('../results/fda/intermediate_triples/fda_triples_product_ndc.tsv', triples_product_ndc)
write_tsv('../results/fda/intermediate_triples/fda_triples_spl_ndc.tsv', triples_spl_ndc)
write_tsv('../results/fda/intermediate_triples/fda_triples_package_ndc.tsv', triples_package_ndc)

In [33]:
triples=[]
for counter in range(1,10):
    print(counter)
    filename='../data/fda/drug-label-000'+str(counter)+'-of-0009.json'
    print(filename)
    
    with open(filename) as ndc_json: 

        data=json.load(ndc_json)
        #Check meta-data
        #print(data['meta'])
        
        for entry in data['results']:
            
            set_id=entry['set_id']
            #print(set_id)
            
            
            klist=entry.keys()
            #print(klist)
            
            if set_id in spl_setid_dict:
   
                #if product_ndc=='0002-8824':
                for k in klist:
                    if k not in ('openfda'):
                        if type(entry[k])== list:
                            for field in entry[k]:
                                triple=[]
                                #marketing_start_date= entry['marketing_start_date']
                                triple.append(str(set_id))
                                triple.append(str(k))
                                triple.append(str(field))
                                triples.append(triple)
                        else:
                            triple=[]
                            triple.append(str(set_id))
                            triple.append(str(k))
                            triple.append(str(entry[k]))
                            triples.append(triple)
                    elif k== 'openfda':
                        openfda=entry['openfda']
                        slist=openfda.keys()
                        
                        for p in slist:
                            if type(openfda[p]) == list:
                                for field in openfda[p]:
                                    triple=[]
                                    triple.append(str(set_id))
                                    triple.append(str(p))
                                    triple.append(str(field))
                                    triples.append(triple)
                            else:
                                triple=[]
                                triple.append(str(set_id))
                                triple.append(str(p))
                                triple.append(str(openfda[p]))
                                triples.append(triple)
                
                #triple.append
            #print("Triples=", triples)
print("Added triples from FDA labeling")

1
../data/fda/drug-label-0001-of-0009.json
2
../data/fda/drug-label-0002-of-0009.json
3
../data/fda/drug-label-0003-of-0009.json
4
../data/fda/drug-label-0004-of-0009.json
5
../data/fda/drug-label-0005-of-0009.json
6
../data/fda/drug-label-0006-of-0009.json
7
../data/fda/drug-label-0007-of-0009.json
8
../data/fda/drug-label-0008-of-0009.json
9
../data/fda/drug-label-0009-of-0009.json
Added triples from FDA labeling


In [34]:
write_tsv('../results/fda/intermediate_triples/fda_triples_spl_labeling.tsv', triples)

In [35]:
triples=[]
products_dict={}
submisions_dict={}
with open('../data/fda/drug-drugsfda-0001-of-0001.json') as drugsfda_json:
    data=json.load(drugsfda_json)
    
    for entry in data['results']:
        
        #product_ndc=entry['product_ndc']
        klist=entry.keys()
        #print(klist)
        if 'application_number' in klist:
            application_no=entry['application_number']
            
            if application_no in application_dict:
      
                for k in klist:
                    if k=='openfda':
                        openfda=entry['openfda']
                        slist=openfda.keys()
                        
                        for p in slist:
                            if type(openfda[p]) == list:
                                for field in openfda[p]:
                                    #if (p=='product_ndc' and field in ndc2seg_list) or (p=='package_ndc' and field in ndc3seg_list) or (p=='spl_set_id' and field in spl_setid_list):
                                    triple=[]
                                    triple.append(str(application_no))
                                    triple.append(str(p))
                                    triple.append(str(field))
                                    triples.append(triple)
                            else:
                                triple=[]
                                triple.append(str(application_no))
                                triple.append(str(p))
                                triple.append(str(openfda[p]))
                                triples.append(triple)
                    elif k  in ('sponsor_name', 'application_number'):
                        triple=[]
                        triple.append(str(application_no))
                        triple.append(str(k))
                        triple.append(str(entry[k]))
                        triples.append(triple)
                    elif k in ('products'):
                        products=entry[k]
                        for prod in products:
                            slist=prod.keys()
                            prod_no=prod['product_number']
                            for p in slist:
                                    
                                if type(prod[p]) == list:
                                    for field in prod[p]:
                                        #if (p=='product_ndc' and field in ndc2seg_list) or (p=='package_ndc' and field in ndc3seg_list) or (p=='spl_set_id' and field in spl_setid_list):
                                        triple=[]
                                        triple.append(str(application_no))
                                        triple.append(str(p))
                                        triple.append(str(field))
                                        #triples.append(triple)
                                else:
                                    triple=[]
                                    triple.append(str(application_no))
                                    triple.append(str(p))
                                    triple.append(str(prod[p]))
                                    
                                if p=='product_number':
                                    triples.append(triple)
                                else:
                                    product_dict_key=str(application_dict[application_no])+"_"+str(prod_no)
                                    if product_dict_key not in products_dict:
                                         products_dict[product_dict_key]=[]
                                         products_dict[product_dict_key].append(triple)
                                    else:
                                        products_dict[product_dict_key].append(triple)
                    elif k == 'submissions':
                        submissions=entry[k]
                        for sub in submissions:
                            slist=sub.keys()
                            #print(slist)
                            sub_date=sub['submission_status_date']
                            for p in slist:
                                    
                                if type(sub[p]) == list:
                                    for field in sub[p]:
                                        #if (p=='product_ndc' and field in ndc2seg_list) or (p=='package_ndc' and field in ndc3seg_list) or (p=='spl_set_id' and field in spl_setid_list):
                                        triple=[]
                                        triple.append(str(application_no))
                                        triple.append(str(p))
                                        triple.append(str(field))
                                        #triples.append(triple)
                                else:
                                    triple=[]
                                    triple.append(str(application_no))
                                    triple.append(str(p))
                                    triple.append(str(sub[p]))
                                    
                                if p=='submission_status_date':
                                    triples.append(triple)
                                else:
                                    submisions_dict_key=str(application_dict[application_no])+"_"+str(sub_date)
                                    if submisions_dict_key not in submisions_dict:
                                         submisions_dict[submisions_dict_key]=[]
                                         submisions_dict[submisions_dict_key].append(triple)
                                    else:
                                        submisions_dict[submisions_dict_key].append(triple)
print("Added triples from FDA DrugsFDA")

Added triples from FDA DrugsFDA


In [36]:
write_tsv('../results/fda/intermediate_triples/fda_triples_application_drugsfda.tsv', triples)   

In [37]:
with open('../data/fda/drug-enforcement-0001-of-0001.json') as ndc_json:
    data=json.load(ndc_json)
    #Check meta-data
    #print(data['meta'])
    triples=[]
    for entry in data['results']:
        if "NDC" in entry['product_description']:
            code = re.search(r'(\d+-?){1,3}$', entry['product_description'])
            #product_ndc=entry['product_description'].split("NDC # ",1)[1]
            if code != None:
                #if str(entry['event_id'])=='67118':
                #print("Event=",entry['event_id'])
            
                #print(code.group())
                #print(product_ndc)
                subj=code.group()
                if (len(str(code.group()))>10) and subj in ndc3seg_dict:
                    
            
                    klist=entry.keys()
                    #print(klist)
                    
                    #if product_ndc=='0002-8824':
                    for k in klist:
                        triple=[]
                        #marketing_start_date= entry['marketing_start_date']
                        triple.append(subj)
                        triple.append(str(k))
                        triple.append(str(entry[k]))
                        triples.append(triple)
print("Added triples from FDA Enforcement")

Added triples from FDA Enforcement


In [38]:
write_tsv('../results/fda/intermediate_triples/fda_triples_package_enforcement.tsv', triples)

In [39]:
with open('../results/rxnorm/intermediate_triples/rxnorm_triples.tsv', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter='\t')
    
    header_list = next(reader)
    print("Header=", header_list)
    
    #umlscui_dict={}
    
    
    for row in reader:
        output_row=[]
    
        #if row[0]=='284460':
        obj=str(row[2])
        #print(obj[:1])
        pred=str(row[1])
        #print("\n")
        #print("Obj=",obj)
        #print("Pred=",pred)
        
        #if obj[:1].isdigit() and pred in ("name", "synonym"):
            #obj="Qty"+obj
        #if obj[:1].isdigit() and pred in ("umlscui"):
            #obj="RXCUI-"+obj
        #elif obj[:1]=='C':
            #print("OBJ IS", obj)
            #print("PRED IS", pred)
            #umlscui_dict[row[0]]=obj
        if pred=="name":
            pred="label"
        elif pred=="tty":
            pred="description"
            obj=tty_dict[obj].replace("_"," ")
        elif pred=="synonym":
            pred="alias"
        
        elif pred in tty_dict.keys():
            pred=tty_dict[pred]
            if obj in qnode_dict_notinwiki.keys():
                obj=qnode_dict_notinwiki[obj]
            elif obj in qnode_dict_inwiki.keys():
                obj=qnode_dict_inwiki[obj]
            else:
                print("Not in Both!")
        elif pred in rela_types_list:
             #  print("GET OUT")
             if obj in qnode_dict_notinwiki.keys():
                obj=qnode_dict_notinwiki[obj]
             elif obj in qnode_dict_inwiki.keys():
                obj=qnode_dict_inwiki[obj]
             else:
                obj="QRX"+obj #temp
                
           
        
        
        
        if pred in ("alias", "label", "description", "suppress"):
            if "'" in obj:
                obj=obj.replace("\'","")
            if obj != '':
                obj="\'"+obj+"\'"+"@en"   #This is hardcoded for now
        if pred in ( "rxcui", "UMLSCUI", "UMLSAUI") or pred in identifier_source_list or pred == 'NDC' or "RXN_" in pred or "ORIG_" in pred:
            obj="\""+obj+"\""
        elif pred =="language":
            obj="en"
        
        
        

        if pred in ("alias","label","description", "rxcui", "UMLSCUI", "SNOMEDCT_US", "MSH", "DRUGBANK", "NDC", "ATC") and str(obj) != '':
            if pred in ("rxcui", "UMLSCUI","SNOMEDCT_US", "MSH", "DRUGBANK", "NDC", "ATC"):
                pred=pred_wikidata_dict[pred]
            if row[0] in qnode_dict_notinwiki:
                output_row.append(str(qnode_dict_notinwiki[row[0]]))
                output_row.append(str(pred))
                output_row.append(str(obj))
            
                output_rows_qrxnode_pnode_rxnorm.append(output_row)
            else:
                output_row.append(str(qnode_dict_inwiki[row[0]]))
                output_row.append(str(pred))
                output_row.append(str(obj))
            
                output_rows_qnode_pnode_rxnorm.append(output_row)
        else:
            if pred not in("language") and str(obj) != '':
                if pred in tty_dict.values():
                    pred_notinwikidata_dict_rxnorm[pred]="PRX_TTY_"+pred
                elif pred in identifier_source_list:
                    pred_notinwikidata_dict_rxnorm[pred]="PRX_ID_"+pred
                elif pred in rela_types_list:
                    pred_notinwikidata_dict_rxnorm[pred]="PRX_REL_"+pred
                else:
                     pred_notinwikidata_dict_rxnorm[pred]="PRX_"+pred
                pred=pred_notinwikidata_dict_rxnorm[pred]
                
                if row[0] in qnode_dict_notinwiki:
                    output_row.append(str(qnode_dict_notinwiki[row[0]]))
                    
                    output_row.append(str(pred))
                    output_row.append(str(obj))
                    output_rows_qrxnode_prxnode_rxnorm.append(output_row)
                else:
                    output_row.append(str(qnode_dict_inwiki[row[0]]))
                    output_row.append(str(pred))
                    output_row.append(str(obj))
                
                    output_rows_qnode_prxnode_rxnorm.append(output_row)

print(output_rows_qrxnode_pnode_rxnorm)

Header= ['node1', 'label', 'node2']
[['QRX317541', 'P31', 'Q28885102'], ['QRX1151131', 'P31', 'Q28885102'], ['QRX1151133', 'P31', 'Q28885102'], ['QRX1656334', 'P31', 'Q28885102'], ['QRX1656335', 'P31', 'Q28885102'], ['QRX1656336', 'P31', 'Q28885102'], ['QRX1656337', 'P31', 'Q28885102'], ['QRX1656338', 'P31', 'Q28885102'], ['QRX1656340', 'P31', 'Q28885102'], ['QRX1656342', 'P31', 'Q28885102'], ['QRX1656343', 'P31', 'Q28885102'], ['QRX1656344', 'P31', 'Q28885102'], ['QRX1656345', 'P31', 'Q28885102'], ['QRX1656346', 'P31', 'Q28885102'], ['QRX1656347', 'P31', 'Q28885102'], ['QRX1656348', 'P31', 'Q28885102'], ['QRX1656349', 'P31', 'Q28885102'], ['QRX1656350', 'P31', 'Q28885102'], ['QRX1656351', 'P31', 'Q28885102'], ['QRX1656352', 'P31', 'Q28885102'], ['QRX1656353', 'P31', 'Q28885102'], ['QRX1656354', 'P31', 'Q28885102'], ['QRX1656355', 'P31', 'Q28885102'], ['QRX1656356', 'P31', 'Q28885102'], ['QRX317541', 'P3345', '"317541"'], ['QRX317541', 'description', "'dose form'@en"], ['QRX317541', 'l

In [40]:
write_tsv("../results/rxnorm/kgtk_triples/nodes/qrxnode_pnode_rxnorm.tsv", output_rows_qrxnode_pnode_rxnorm)
write_tsv("../results/rxnorm/kgtk_triples/nodes/qrxnode_prxnode_rxnorm.tsv", output_rows_qrxnode_prxnode_rxnorm)
  
write_tsv("../results/rxnorm/kgtk_triples/nodes/qnode_pnode_rxnorm.tsv", output_rows_qnode_pnode_rxnorm)
write_tsv("../results/rxnorm/kgtk_triples/nodes/qnode_prxnode_rxnorm.tsv", output_rows_qnode_prxnode_rxnorm)

In [41]:
for x in pred_notinwikidata_dict_rxnorm.keys():
    
    print(x)
    
    output_row=[]
    pred_value=pred_notinwikidata_dict_rxnorm[x]
    output_row.append(pred_value)
    output_row.append("label")
    output_row.append("\'"+x+"\'"+"@en")
    output_rows_prxnode_rxnorm.append(output_row)
    output_rows_prxnode_edges_rxnorm.append(output_row)
    
    output_row=[]
    output_row.append(pred_value)
    output_row.append("description")
    output_row.append("\'"+x+"\'"+"@en")
    output_rows_prxnode_rxnorm.append(output_row)
    output_rows_prxnode_edges_rxnorm.append(output_row)
    
    output_row=[]
    output_row.append(pred_value)
    output_row.append("data_type")
    if ("PRX_REL_" in pred_value or "PRX_TTY_" in pred_value):
        output_row.append("\""+"item"+"\"")
    elif ("PRX_ID_" in pred_value):
        output_row.append("\""+"external-identifier"+"\"")
    elif ("PRX_" in pred_value):
        output_row.append("\""+"string"+"\"")
    #output_row.append("\"String\"")
    output_rows_prxnode_rxnorm.append(output_row)
    output_rows_prxnode_datatype_rxnorm.append(output_row)

USP
GS
VANDF
MTHSPL
NDDF
MMSL
suppress
MMX
inverse_isa
has_ingredient
dose_form_of
ingredient_of
isa
doseformgroup_of
has_tradename
tradename_of
has_dose_form
constitutes
ingredients_of
has_doseformgroup
has_ingredients
consists_of
part_of
has_part
NDC 3 Segment
NDC 2 Segment
RXN_AVAILABLE_STRENGTH
RXN_BOSS_AM
SPL_SET_ID
DrugsFDA Application Number
RXN_BOSS_STRENGTH_NUM_VALUE
RXN_BOSS_STRENGTH_DENOM_UNIT
RXN_BOSS_STRENGTH_DENOM_VALUE
RXN_BOSS_STRENGTH_NUM_UNIT
RXN_BOSS_AI
RXTERM_FORM
RXN_STRENGTH
RXN_HUMAN_DRUG
RXN_BOSS_FROM
ORIG_SOURCE
ORIG_CODE
RXN_BN_CARDINALITY
UMLSAUI


In [42]:
write_tsv("../results/rxnorm/kgtk_triples/properties/prxnode_rxnorm.tsv", output_rows_prxnode_rxnorm)
write_tsv("../results/rxnorm/kgtk_triples/properties/prxnode_edges_rxnorm.tsv", output_rows_prxnode_edges_rxnorm)
write_tsv("../results/rxnorm/kgtk_triples/properties/prxnode_datatype_rxnorm.tsv", output_rows_prxnode_datatype_rxnorm)

In [43]:
output_rows_qnode_pnode_fda=[]
output_rows_qnode_prxnode_fda=[]
output_rows_qrxnode_pnode_fda=[]
output_rows_qrxnode_prxnode_fda=[]
output_rows_prxnode_fda=[]
output_rows_prxnode_edges_fda=[]
output_rows_prxnode_datatype_fda=[]

In [44]:
pred_notinwikidata_dict_fda={}

In [45]:
with open('../results/fda/intermediate_triples/fda_triples_product_ndc.tsv', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter='\t')
    
    header_list = next(reader)
    print("Header=", header_list)
    
    #umlscui_dict={}
    
   
    for row in reader:
        output_row=[]
        
        obj=str(row[2])
        if "'" in obj:
            obj=obj.replace("\'","\\'")

            
        #print(obj[:1])
        pred=str(row[1])
        if pred== "package_ndc":
            pred="NDC 3 Segment"
        elif pred== "spl_set_id":
            pred="SPL_SET_ID"
        elif pred=="rxcui":
            pred="Related RXNorm"
        
       
        subj=ndc2seg_dict[str(row[0])]
        
        
        if "date" in pred:
            obj="^"+'-'.join([obj[:4], obj[4:6], obj[6:]])
        elif pred in ("NDC 3 Segment", "SPL_SET_ID", "spl_id", "product_id"):
            obj="\""+obj+"\""
        elif pred not in ("finished"):
            obj="\'"+obj+"\'"+"@en" 
            
        if subj in qnode_dict_notinwiki and pred in pred_wikidata_dict:
            subj=qnode_dict_notinwiki[subj]
            output_row.append(str(subj))
            output_row.append(str(pred_wikidata_dict[pred]))
            output_row.append(str(obj))
        
            output_rows_qrxnode_pnode_fda.append(output_row)
        elif subj in qnode_dict_notinwiki and pred not in pred_wikidata_dict:
            subj=qnode_dict_notinwiki[subj]
            output_row.append(str(subj))
            if pred in identifier_source_list:
                pred_notinwikidata_dict_fda[pred]="PRX_ID_"+pred
            else:
                pred_notinwikidata_dict_fda[pred]="PRX_FDA_"+pred
            pred=pred_notinwikidata_dict_fda[pred]
            output_row.append(str(pred))
            output_row.append(str(obj))
        
            output_rows_qrxnode_prxnode_fda.append(output_row)
            
        elif subj not in qnode_dict_notinwiki and pred in pred_wikidata_dict:
            subj=qnode_dict_inwiki[subj]
            output_row.append(str(subj))
            output_row.append(str(pred_wikidata_dict[pred]))
            output_row.append(str(obj))
        
            output_rows_qnode_pnode_fda.append(output_row)
        elif subj not in qnode_dict_notinwiki and pred not in pred_wikidata_dict:
            subj=qnode_dict_inwiki[subj]
            output_row.append(str(subj))
            if pred in identifier_source_list:
                pred_notinwikidata_dict_fda[pred]="PRX_ID_"+pred
            else:
                pred_notinwikidata_dict_fda[pred]="PRX_FDA_"+pred
            pred=pred_notinwikidata_dict_fda[pred]
            output_row.append(str(pred))
            output_row.append(str(obj))
        
            output_rows_qnode_prxnode_fda.append(output_row)

Header= ['node1', 'label', 'node2']


In [46]:
with open('../results/fda/intermediate_triples/fda_triples_package_ndc.tsv', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter='\t')
    
    header_list = next(reader)
    print("Header=", header_list)
    
    #umlscui_dict={}
    
   
    for row in reader:
        output_row=[]
        
        obj=str(row[2])
        if "'" in obj:
            obj=obj.replace("\'","\\'")
        #print(obj[:1])
        pred=str(row[1])
        if pred== "package_ndc":
            pred="NDC 3 Segment"
        elif pred== "spl_set_id":
            pred="SPL_SET_ID"
        elif pred=="rxcui":
            pred="Related RXNorm"
            
        subj=ndc3seg_dict[str(row[0])]
        
        if "date" in pred:
            obj="^"+'-'.join([obj[:4], obj[4:6], obj[6:]])
        elif pred in ("NDC 3 Segment", "SPL_SET_ID", "spl_id", "product_id"):
            obj="\""+obj+"\""
        elif pred not in ("sample"):
            obj="\'"+obj+"\'"+"@en" 
        
        if subj in qnode_dict_notinwiki and pred in pred_wikidata_dict:
            subj=qnode_dict_notinwiki[subj]
            output_row.append(str(subj))
            output_row.append(str(pred_wikidata_dict[pred]))
            output_row.append(str(obj))
        
            output_rows_qrxnode_pnode_fda.append(output_row)
        elif subj in qnode_dict_notinwiki and pred not in pred_wikidata_dict:
            subj=qnode_dict_notinwiki[subj]
            output_row.append(str(subj))
            if pred in identifier_source_list:
                pred_notinwikidata_dict_fda[pred]="PRX_ID_"+pred
            else:
                pred_notinwikidata_dict_fda[pred]="PRX_FDA_"+pred
            pred=pred_notinwikidata_dict_fda[pred]
            output_row.append(str(pred))
            output_row.append(str(obj))
        
            output_rows_qrxnode_prxnode_fda.append(output_row)
            
        elif subj not in qnode_dict_notinwiki and pred in pred_wikidata_dict:
            subj=qnode_dict_inwiki[subj]
            output_row.append(str(subj))
            output_row.append(str(pred_wikidata_dict[pred]))
            output_row.append(str(obj))
        
            output_rows_qnode_pnode_fda.append(output_row)
        elif subj not in qnode_dict_notinwiki and pred not in pred_wikidata_dict:
            subj=qnode_dict_inwiki[subj]
            output_row.append(str(subj))
            if pred in identifier_source_list:
                pred_notinwikidata_dict_fda[pred]="PRX_ID_"+pred
            else:
                pred_notinwikidata_dict_fda[pred]="PRX_FDA_"+pred
            pred=pred_notinwikidata_dict_fda[pred]
            output_row.append(str(pred))
            output_row.append(str(obj))
        
            output_rows_qnode_prxnode_fda.append(output_row)

Header= ['node1', 'label', 'node2']


In [47]:
with open('../results/fda/intermediate_triples/fda_triples_spl_ndc.tsv', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter='\t')
    
    header_list = next(reader)
    print("Header=", header_list)
    
    #umlscui_dict={}
    
   
    for row in reader:
        output_row=[]
        
        obj=str(row[2])
        if "'" in obj:
            obj=obj.replace("\'","\\'")
        #print(obj[:1])
        pred=str(row[1])
        if pred== "package_ndc":
            pred="NDC 3 Segment"
        elif pred== "spl_set_id":
            pred="SPL_SET_ID"
        elif pred=="rxcui":
            pred="Related RXNorm"
        
        
        subj=spl_setid_dict[str(row[0])]
        if "date" in pred:
            obj="^"+'-'.join([obj[:4], obj[4:6], obj[6:]])
        elif pred in ("NDC 3 Segment", "SPL_SET_ID", "spl_id", "product_id", "Related RXNorm", "unii", "upc"):
            obj="\""+obj+"\""
        else:
            obj="\'"+obj+"\'"+"@en" 
        if subj in qnode_dict_notinwiki and pred in pred_wikidata_dict:
            subj=qnode_dict_notinwiki[subj]
            output_row.append(str(subj))
            output_row.append(str(pred_wikidata_dict[pred]))
            output_row.append(str(obj))
        
            output_rows_qrxnode_pnode_fda.append(output_row)
        elif subj in qnode_dict_notinwiki and pred not in pred_wikidata_dict:
            subj=qnode_dict_notinwiki[subj]
            output_row.append(str(subj))
            if pred in identifier_source_list:
                pred_notinwikidata_dict_fda[pred]="PRX_ID_"+pred
            else:
                pred_notinwikidata_dict_fda[pred]="PRX_FDA_"+pred
            pred=pred_notinwikidata_dict_fda[pred]
            output_row.append(str(pred))
            output_row.append(str(obj))
        
            output_rows_qrxnode_prxnode_fda.append(output_row)
            
        elif subj not in qnode_dict_notinwiki and pred in pred_wikidata_dict:
            subj=qnode_dict_inwiki[subj]
            output_row.append(str(subj))
            output_row.append(str(pred_wikidata_dict[pred]))
            output_row.append(str(obj))
        
            output_rows_qnode_pnode_fda.append(output_row)
        elif subj not in qnode_dict_notinwiki and pred not in pred_wikidata_dict:
            subj=qnode_dict_inwiki[subj]
            output_row.append(str(subj))
            if pred in identifier_source_list:
                pred_notinwikidata_dict_fda[pred]="PRX_ID_"+pred
            else:
                pred_notinwikidata_dict_fda[pred]="PRX_FDA_"+pred
            pred=pred_notinwikidata_dict_fda[pred]
            output_row.append(str(pred))
            output_row.append(str(obj))
        
            output_rows_qnode_prxnode_fda.append(output_row)

Header= ['node1', 'label', 'node2']


In [48]:
with open('../results/fda/intermediate_triples/fda_triples_spl_labeling.tsv', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter='\t')
    
    header_list = next(reader)
    print("Header=", header_list)
    
    #umlscui_dict={}
    
   
    for row in reader:
        output_row=[]
        
        obj=str(row[2])
        if "'" in obj:
            obj=obj.replace("\'","\\'")
        #print(obj[:1])
        pred=str(row[1])
        if pred== "package_ndc":
            pred="NDC 3 Segment"
        elif pred== "spl_set_id":
            pred="SPL_SET_ID"
        elif pred=="rxcui":
            pred="Related RXNorm"
        
        subj=spl_setid_dict[str(row[0])]
        if "date" in pred:
            obj="^"+'-'.join([obj[:4], obj[4:6], obj[6:]])
        elif pred in ("NDC 3 Segment", "SPL_SET_ID", "spl_id", "product_id", "Related RXNorm", "unii", "upc"):
            obj="\""+obj+"\""
        else:
            obj="\'"+obj+"\'"+"@en" 
        if subj in qnode_dict_notinwiki and pred in pred_wikidata_dict:
            subj=qnode_dict_notinwiki[subj]
            output_row.append(str(subj))
            output_row.append(str(pred_wikidata_dict[pred]))
            output_row.append(str(obj))
        
            output_rows_qrxnode_pnode_fda.append(output_row)
        elif subj in qnode_dict_notinwiki and pred not in pred_wikidata_dict:
            subj=qnode_dict_notinwiki[subj]
            output_row.append(str(subj))
            if pred in identifier_source_list:
                pred_notinwikidata_dict_fda[pred]="PRX_ID_"+pred
            else:
                pred_notinwikidata_dict_fda[pred]="PRX_FDA_"+pred
            pred=pred_notinwikidata_dict_fda[pred]
            output_row.append(str(pred))
            output_row.append(str(obj))
        
            output_rows_qrxnode_prxnode_fda.append(output_row)
            
        elif subj not in qnode_dict_notinwiki and pred in pred_wikidata_dict:
            subj=qnode_dict_inwiki[subj]
            output_row.append(str(subj))
            output_row.append(str(pred_wikidata_dict[pred]))
            output_row.append(str(obj))
        
            output_rows_qnode_pnode_fda.append(output_row)
        elif subj not in qnode_dict_notinwiki and pred not in pred_wikidata_dict:
            subj=qnode_dict_inwiki[subj]
            output_row.append(str(subj))
            if pred in identifier_source_list:
                pred_notinwikidata_dict_fda[pred]="PRX_ID_"+pred
            else:
                pred_notinwikidata_dict_fda[pred]="PRX_FDA_"+pred
            pred=pred_notinwikidata_dict_fda[pred]
            output_row.append(str(pred))
            output_row.append(str(obj))
        
            output_rows_qnode_prxnode_fda.append(output_row)

Header= ['node1', 'label', 'node2']


In [49]:
with open('../results/fda/intermediate_triples/fda_triples_application_drugsfda.tsv', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter='\t')
    
    header_list = next(reader)
    print("Header=", header_list)
    
    #umlscui_dict={}
    
   
    for row in reader:
        output_row=[]
        
        obj=str(row[2])
        if "'" in obj:
            obj=obj.replace("\'","\\'")
        #print(obj[:1])
        pred=str(row[1])
        if pred== "package_ndc":
            pred="NDC 3 Segment"
        elif pred== "spl_set_id":
            pred="SPL_SET_ID"
        elif pred=="rxcui":
            pred="Related RXNorm"
        
        subj=application_dict[str(row[0])]
        if "date" in pred:
            obj="^"+'-'.join([obj[:4], obj[4:6], obj[6:]])
        elif pred in ("NDC 3 Segment", "SPL_SET_ID", "spl_id", "product_id", "Related RXNorm", "unii", "upc"):
            obj="\""+obj+"\""
        else:
            obj="\'"+obj+"\'"+"@en" 
        if subj in qnode_dict_notinwiki and pred in pred_wikidata_dict:
            subj=qnode_dict_notinwiki[subj]
            output_row.append(str(subj))
            output_row.append(str(pred_wikidata_dict[pred]))
            output_row.append(str(obj))
        
            output_rows_qrxnode_pnode_fda.append(output_row)
        elif subj in qnode_dict_notinwiki and pred not in pred_wikidata_dict:
            subj=qnode_dict_notinwiki[subj]
            output_row.append(str(subj))
            if pred in identifier_source_list:
                pred_notinwikidata_dict_fda[pred]="PRX_ID_"+pred
            else:
                pred_notinwikidata_dict_fda[pred]="PRX_FDA_"+pred
            pred=pred_notinwikidata_dict_fda[pred]
            output_row.append(str(pred))
            output_row.append(str(obj))
        
            output_rows_qrxnode_prxnode_fda.append(output_row)
            
        elif subj not in qnode_dict_notinwiki and pred in pred_wikidata_dict:
            subj=qnode_dict_inwiki[subj]
            output_row.append(str(subj))
            output_row.append(str(pred_wikidata_dict[pred]))
            output_row.append(str(obj))
        
            output_rows_qnode_pnode_fda.append(output_row)
        elif subj not in qnode_dict_notinwiki and pred not in pred_wikidata_dict:
            subj=qnode_dict_inwiki[subj]
            output_row.append(str(subj))
            if pred in identifier_source_list:
                pred_notinwikidata_dict_fda[pred]="PRX_ID_"+pred
            else:
                pred_notinwikidata_dict_fda[pred]="PRX_FDA_"+pred
            pred=pred_notinwikidata_dict_fda[pred]
            output_row.append(str(pred))
            output_row.append(str(obj))
        
            output_rows_qnode_prxnode_fda.append(output_row)

Header= ['node1', 'label', 'node2']


In [50]:
with open('../results/fda/intermediate_triples/fda_triples_package_enforcement.tsv', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter='\t')
    
    header_list = next(reader)
    print("Header=", header_list)
    
    #umlscui_dict={}
    
   
    for row in reader:
        output_row=[]
        
        obj=str(row[2])
        if "'" in obj:
            obj=obj.replace("\'","\\'")
        #print(obj[:1])
        pred=str(row[1])
        if pred== "package_ndc":
            pred="NDC 3 Segment"
        elif pred== "spl_set_id":
            pred="SPL_SET_ID"
        elif pred=="rxcui":
            pred="Related RXNorm"
        
        subj=ndc3seg_dict[str(row[0])]
        if "date" in pred:
            obj="^"+'-'.join([obj[:4], obj[4:6], obj[6:]])
        elif pred in ("NDC 3 Segment", "SPL_SET_ID", "spl_id", "product_id", "Related RXNorm", "unii", "upc", "event_id"):
            obj="\""+obj+"\""
        else:
            obj="\'"+obj+"\'"+"@en" 
        if subj in qnode_dict_notinwiki and pred in pred_wikidata_dict:
            subj=qnode_dict_notinwiki[subj]
            output_row.append(str(subj))
            output_row.append(str(pred_wikidata_dict[pred]))
            output_row.append(str(obj))
        
            output_rows_qrxnode_pnode_fda.append(output_row)
        elif subj in qnode_dict_notinwiki and pred not in pred_wikidata_dict:
            subj=qnode_dict_notinwiki[subj]
            output_row.append(str(subj))
            if pred in identifier_source_list:
                pred_notinwikidata_dict_fda[pred]="PRX_ID_"+pred
            else:
                pred_notinwikidata_dict_fda[pred]="PRX_FDA_"+pred
            pred=pred_notinwikidata_dict_fda[pred]
            output_row.append(str(pred))
            output_row.append(str(obj))
        
            output_rows_qrxnode_prxnode_fda.append(output_row)
            
        elif subj not in qnode_dict_notinwiki and pred in pred_wikidata_dict:
            subj=qnode_dict_inwiki[subj]
            output_row.append(str(subj))
            output_row.append(str(pred_wikidata_dict[pred]))
            output_row.append(str(obj))
        
            output_rows_qnode_pnode_fda.append(output_row)
        elif subj not in qnode_dict_notinwiki and pred not in pred_wikidata_dict:
            subj=qnode_dict_inwiki[subj]
            output_row.append(str(subj))
            if pred in identifier_source_list:
                pred_notinwikidata_dict_fda[pred]="PRX_ID_"+pred
            else:
                pred_notinwikidata_dict_fda[pred]="PRX_FDA_"+pred
            pred=pred_notinwikidata_dict_fda[pred]
            output_row.append(str(pred))
            output_row.append(str(obj))
        
            output_rows_qnode_prxnode_fda.append(output_row)

Header= ['node1', 'label', 'node2']


In [51]:
write_tsv('../results/fda/kgtk_triples/nodes/qrxnode_prxnode_fda.tsv', output_rows_qrxnode_prxnode_fda)
write_tsv('../results/fda/kgtk_triples/nodes/qnode_prxnode_fda.tsv', output_rows_qnode_prxnode_fda)

write_tsv('../results/fda/kgtk_triples/nodes/qrxnode_pnode_fda.tsv', output_rows_qrxnode_pnode_fda)
write_tsv('../results/fda/kgtk_triples/nodes/qnode_pnode_fda.tsv', output_rows_qnode_pnode_fda)

In [52]:
for x in pred_notinwikidata_dict_fda.keys():
    
    print(x)
    
    output_row=[]
    pred_value=pred_notinwikidata_dict_fda[x]
    output_row.append(pred_value)
    output_row.append("label")
    output_row.append("\'"+x+"\'"+"@en")
    output_rows_prxnode_fda.append(output_row)
    output_rows_prxnode_edges_fda.append(output_row)
    
    output_row=[]
    output_row.append(pred_value)
    output_row.append("description")
    output_row.append("\'"+x+"\'"+"@en")
    output_rows_prxnode_fda.append(output_row)
    output_rows_prxnode_edges_fda.append(output_row)
    
    output_row=[]
    output_row.append(pred_value)
    output_row.append("data_type")
    if ("PRX_ID_" in pred_value):
        output_row.append("\""+"external-identifier"+"\"")
    elif ("PRX_FDA_" in pred_value):
        output_row.append("\""+"string"+"\"")
    #output_row.append("\"String\"")
    output_rows_prxnode_fda.append(output_row)
    output_rows_prxnode_datatype_fda.append(output_row)

marketing_start_date
product_type
product_id
marketing_category
route
generic_name
brand_name
brand_name_base
labeler_name
active_ingredients
finished
spl_id
pharm_class
application_number
listing_expiration_date
dosage_form
sample
description
marketing_end_date
is_original_packager
upc
Related RXNorm
package_label_principal_display_panel
drug_interactions
id
indications_and_usage
spl_unclassified_section
SPL_SET_ID
product_ndc
substance_name
NDC 3 Segment
contraindications
version
adverse_reactions
information_for_patients
dosage_forms_and_strengths
clinical_studies_table
dosage_and_administration
effective_time
spl_product_data_elements
clinical_pharmacology
clinical_studies
use_in_specific_populations
nonclinical_toxicology
boxed_warning
overdosage
warnings_and_cautions
how_supplied
set_id
carcinogenesis_and_mutagenesis_and_impairment_of_fertility
pregnancy
pharmacokinetics
labor_and_delivery
spl_patient_package_insert
pediatric_use
mechanism_of_action
geriatric_use
adverse_reaction

In [53]:
write_tsv("../results/fda/kgtk_triples/properties/prxnode_fda.tsv", output_rows_prxnode_fda)
write_tsv("../results/fda/kgtk_triples/properties/prxnode_edges_fda.tsv", output_rows_prxnode_edges_fda)
write_tsv("../results/fda/kgtk_triples/properties/prxnode_datatype_fda.tsv", output_rows_prxnode_datatype_fda)

In [54]:
!kgtk compact -i ../results/rxnorm/kgtk_triples/nodes/qrxnode_pnode_rxnorm.tsv -o ../results/rxnorm/kgtk_triples_compact/nodes/qrxnode_pnode_rxnorm_compact.tsv
!kgtk compact -i ../results/rxnorm/kgtk_triples/nodes/qrxnode_prxnode_rxnorm.tsv -o ../results/rxnorm/kgtk_triples_compact/nodes/qrxnode_prxnode_rxnorm_compact.tsv
!kgtk compact -i ../results/rxnorm/kgtk_triples/nodes/qnode_pnode_rxnorm.tsv -o ../results/rxnorm/kgtk_triples_compact/nodes/qnode_pnode_rxnorm_compact.tsv
!kgtk compact -i ../results/rxnorm/kgtk_triples/nodes/qnode_prxnode_rxnorm.tsv -o ../results/rxnorm/kgtk_triples_compact/nodes/qnode_prxnode_rxnorm_compact.tsv

In [55]:
!kgtk compact -i ../results/fda/kgtk_triples/nodes/qrxnode_pnode_fda.tsv -o ../results/fda/kgtk_triples_compact/nodes/qrxnode_pnode_fda_compact.tsv
!kgtk compact -i ../results/fda/kgtk_triples/nodes/qrxnode_prxnode_fda.tsv -o ../results/fda/kgtk_triples_compact/nodes/qrxnode_prxnode_fda_compact.tsv
!kgtk compact -i ../results/fda/kgtk_triples/nodes/qnode_pnode_fda.tsv -o ../results/fda/kgtk_triples_compact/nodes/qnode_pnode_fda_compact.tsv
!kgtk compact -i ../results/fda/kgtk_triples/nodes/qnode_prxnode_fda.tsv -o ../results/fda/kgtk_triples_compact/nodes/qnode_prxnode_fda_compact.tsv

In [56]:
!kgtk compact -i ../results/rxnorm/kgtk_triples/properties/prxnode_edges_rxnorm.tsv -o ../results/rxnorm/kgtk_triples_compact/properties/prxnode_edges_rxnorm_compact.tsv

In [57]:
!kgtk compact -i ../results/fda/kgtk_triples/properties/prxnode_edges_fda.tsv -o ../results/fda/kgtk_triples_compact/properties/prxnode_edges_fda_compact.tsv

In [58]:
!kgtk add-id -i ../results/rxnorm/kgtk_triples_compact/nodes/qrxnode_pnode_rxnorm_compact.tsv -o ../results/rxnorm/kgtk_triples_id/nodes/qrxnode_pnode_rxnorm_id.tsv --id-style node1-label-node2-num
!kgtk add-id -i ../results/rxnorm/kgtk_triples_compact/nodes/qrxnode_prxnode_rxnorm_compact.tsv -o ../results/rxnorm/kgtk_triples_id/nodes/qrxnode_prxnode_rxnorm_id.tsv --id-style node1-label-node2-num
!kgtk add-id -i ../results/rxnorm/kgtk_triples_compact/nodes/qnode_pnode_rxnorm_compact.tsv -o ../results/rxnorm/kgtk_triples_id/nodes/qnode_pnode_rxnorm_id.tsv --id-style node1-label-node2-num
!kgtk add-id -i ../results/rxnorm/kgtk_triples_compact/nodes/qnode_prxnode_rxnorm_compact.tsv -o ../results/rxnorm/kgtk_triples_id/nodes/qnode_prxnode_rxnorm_id.tsv --id-style node1-label-node2-num

In [59]:
!kgtk add-id -i ../results/fda/kgtk_triples_compact/nodes/qrxnode_pnode_fda_compact.tsv -o ../results/fda/kgtk_triples_id/nodes/qrxnode_pnode_fda_id.tsv --id-style node1-label-node2-num
!kgtk add-id -i ../results/fda/kgtk_triples_compact/nodes/qrxnode_prxnode_fda_compact.tsv -o ../results/fda/kgtk_triples_id/nodes/qrxnode_prxnode_fda_id.tsv --id-style node1-label-node2-num
!kgtk add-id -i ../results/fda/kgtk_triples_compact/nodes/qnode_pnode_fda_compact.tsv -o ../results/fda/kgtk_triples_id/nodes/qnode_pnode_fda_id.tsv --id-style node1-label-node2-num
!kgtk add-id -i ../results/fda/kgtk_triples_compact/nodes/qnode_prxnode_fda_compact.tsv -o ../results/fda/kgtk_triples_id/nodes/qnode_prxnode_fda_id.tsv --id-style node1-label-node2-num

In [60]:
!kgtk add-id -i ../results/rxnorm/kgtk_triples_compact/properties/prxnode_edges_rxnorm_compact.tsv -o ../results/rxnorm/kgtk_triples_id/properties/prxnode_edges_rxnorm_id.tsv --id-style node1-label-node2-num

In [61]:
!kgtk add-id -i ../results/fda/kgtk_triples_compact/properties/prxnode_edges_fda_compact.tsv -o ../results/fda/kgtk_triples_id/properties/prxnode_edges_fda_id.tsv --id-style node1-label-node2-num

In [62]:
!kgtk validate -i ../results/rxnorm/kgtk_triples_id/nodes/qrxnode_pnode_rxnorm_id.tsv -v
!kgtk validate -i ../results/rxnorm/kgtk_triples_id/nodes/qrxnode_prxnode_rxnorm_id.tsv -v
!kgtk validate -i ../results/rxnorm/kgtk_triples_id/nodes/qnode_pnode_rxnorm_id.tsv -v
!kgtk validate -i ../results/rxnorm/kgtk_triples_id/nodes/qnode_prxnode_rxnorm_id.tsv -v


Validating '../results/rxnorm/kgtk_triples_id/nodes/qrxnode_pnode_rxnorm_id.tsv'
KgtkReader: File_path.suffix: .tsv
KgtkReader: reading file ../results/rxnorm/kgtk_triples_id/nodes/qrxnode_pnode_rxnorm_id.tsv
header: node1	label	node2	id
node1 column found, this is a KGTK edge file
KgtkReader: Special columns: node1=0 label=1 node2=2 id=3
KgtkReader: Reading an edge file.
Validated 154 data lines

Validating '../results/rxnorm/kgtk_triples_id/nodes/qrxnode_prxnode_rxnorm_id.tsv'
KgtkReader: File_path.suffix: .tsv
KgtkReader: reading file ../results/rxnorm/kgtk_triples_id/nodes/qrxnode_prxnode_rxnorm_id.tsv
header: node1	label	node2	id
node1 column found, this is a KGTK edge file
KgtkReader: Special columns: node1=0 label=1 node2=2 id=3
KgtkReader: Reading an edge file.
Validated 37405 data lines

Validating '../results/rxnorm/kgtk_triples_id/nodes/qnode_pnode_rxnorm_id.tsv'
KgtkReader: File_path.suffix: .tsv
KgtkReader: reading file ../results/rxnorm/kgtk_triples_id/nodes/qnode_pnode_

In [63]:
!kgtk validate -i ../results/fda/kgtk_triples_id/nodes/qrxnode_pnode_fda_id.tsv -v
!kgtk validate -i ../results/fda/kgtk_triples_id/nodes/qrxnode_prxnode_fda_id.tsv -v
!kgtk validate -i ../results/fda/kgtk_triples_id/nodes/qnode_pnode_fda_id.tsv -v
!kgtk validate -i ../results/fda/kgtk_triples_id/nodes/qnode_prxnode_fda_id.tsv -v


Validating '../results/fda/kgtk_triples_id/nodes/qrxnode_pnode_fda_id.tsv'
KgtkReader: File_path.suffix: .tsv
KgtkReader: reading file ../results/fda/kgtk_triples_id/nodes/qrxnode_pnode_fda_id.tsv
header: node1	label	node2	id
node1 column found, this is a KGTK edge file
KgtkReader: Special columns: node1=0 label=1 node2=2 id=3
KgtkReader: Reading an edge file.
Validated 4 data lines

Validating '../results/fda/kgtk_triples_id/nodes/qrxnode_prxnode_fda_id.tsv'
KgtkReader: File_path.suffix: .tsv
KgtkReader: reading file ../results/fda/kgtk_triples_id/nodes/qrxnode_prxnode_fda_id.tsv
header: node1	label	node2	id
node1 column found, this is a KGTK edge file
KgtkReader: Special columns: node1=0 label=1 node2=2 id=3
KgtkReader: Reading an edge file.
Validated 99 data lines

Validating '../results/fda/kgtk_triples_id/nodes/qnode_pnode_fda_id.tsv'
KgtkReader: File_path.suffix: .tsv
KgtkReader: reading file ../results/fda/kgtk_triples_id/nodes/qnode_pnode_fda_id.tsv
header: node1	label	node2	i

In [64]:
!kgtk validate -i ../results/rxnorm/kgtk_triples_id/properties/prxnode_edges_rxnorm_id.tsv -v


Validating '../results/rxnorm/kgtk_triples_id/properties/prxnode_edges_rxnorm_id.tsv'
KgtkReader: File_path.suffix: .tsv
KgtkReader: reading file ../results/rxnorm/kgtk_triples_id/properties/prxnode_edges_rxnorm_id.tsv
header: node1	label	node2	id
node1 column found, this is a KGTK edge file
KgtkReader: Special columns: node1=0 label=1 node2=2 id=3
KgtkReader: Reading an edge file.
Validated 86 data lines


In [65]:
!kgtk validate -i ../results/fda/kgtk_triples_id/properties/prxnode_edges_fda_id.tsv -v


Validating '../results/fda/kgtk_triples_id/properties/prxnode_edges_fda_id.tsv'
KgtkReader: File_path.suffix: .tsv
KgtkReader: reading file ../results/fda/kgtk_triples_id/properties/prxnode_edges_fda_id.tsv
header: node1	label	node2	id
node1 column found, this is a KGTK edge file
KgtkReader: Special columns: node1=0 label=1 node2=2 id=3
KgtkReader: Reading an edge file.
Validated 158 data lines


In [66]:
output_rows_qrxnode_prxnode_fda_qualifier=[]
output_rows_qnode_prxnode_fda_qualifier=[]
pred_notinwikidata_dict_fda_qualifier={}

with open('../results/fda/kgtk_triples_id/nodes/qrxnode_prxnode_fda_id.tsv', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter='\t')
    
    header_list = next(reader)
    print("Header=", header_list)
    
    #umlscui_dict={}
    print(submisions_dict)
    print(products_dict)
    print(active_ingredients_dict)
   
    for row in reader:
        
        if row[1] in ('PRX_FDA_product_number','PRX_FDA_submission_status_date', 'PRX_FDA_active_ingredients'):
          print(row[0], row[2])
          rxnode=row[0].split('QRX',1)[1]
          if row[1]=='PRX_FDA_product_number':
            prodno=row[2].split('@',1)[0].replace("'","")
            rxnode_key=rxnode+"_"+prodno
            print(rxnode_key)
            if rxnode_key in products_dict:
              product_associated_triples=products_dict[rxnode_key]
              for val in product_associated_triples:
                output_row=[]
                output_row.append(row[3])
                
                pred=val[1]
                if pred in identifier_source_list:
                  pred_notinwikidata_dict_fda_qualifier[pred]="PRX_ID_"+pred
                else:
                  pred_notinwikidata_dict_fda_qualifier[pred]="PRX_FDA_"+pred 
                
                pred=pred_notinwikidata_dict_fda_qualifier[pred]
                output_row.append(pred)
                output_row.append("\""+val[2]+"\"")
                output_rows_qrxnode_prxnode_fda_qualifier.append(output_row)
          elif row[1]=='PRX_FDA_submission_status_date':
            prodno=row[2].split('^',1)[1].replace("-","")
            rxnode_key=rxnode+"_"+prodno
            print(rxnode_key)
            if rxnode_key in submisions_dict:
              sub_associated_triples=submisions_dict[rxnode_key]
              for val in product_associated_triples:
                output_row=[]
                output_row.append(row[3])
                pred=val[1]
                if pred in identifier_source_list:
                  pred_notinwikidata_dict_fda_qualifier[pred]="PRX_ID_"+pred
                else:
                  pred_notinwikidata_dict_fda_qualifier[pred]="PRX_FDA_"+pred 
                
                pred=pred_notinwikidata_dict_fda_qualifier[pred]
                output_row.append(pred)
                output_row.append("\""+val[2]+"\"")
                output_rows_qrxnode_prxnode_fda_qualifier.append(output_row)
          else:
            prodno=row[2].split('@',1)[0].replace("'","")
            rxnode_key=rxnode+"_"+prodno
            print(rxnode_key)
            if rxnode_key in active_ingredients_dict:
              val=active_ingredients_dict[rxnode_key]
              
              output_row=[]
              output_row.append(row[3])
              pred='strength'
              if pred in identifier_source_list:
                pred_notinwikidata_dict_fda_qualifier[pred]="PRX_ID_"+pred
              else:
                pred_notinwikidata_dict_fda_qualifier[pred]="PRX_FDA_"+pred 
              
              pred=pred_notinwikidata_dict_fda_qualifier[pred]
              output_row.append(pred)
              output_row.append("\""+val+"\"")
              output_rows_qrxnode_prxnode_fda_qualifier.append(output_row)

print(output_rows_qrxnode_prxnode_fda_qualifier)

with open('../results/fda/kgtk_triples_id/nodes/qnode_prxnode_fda_id.tsv', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter='\t')
    
    header_list = next(reader)
    print("Header=", header_list)
    
    #umlscui_dict={}
    #print(submisions_dict)
    #print(products_dict)
   
    for row in reader:
        
        if row[1] in ('PRX_FDA_product_number','PRX_FDA_submission_status_date', 'PRX_FDA_active_ingredients'):
          print(row[0], row[2])
          rxnode=get_key(row[0],qnode_dict_inwiki)
          if row[1]=='PRX_FDA_product_number':
            prodno=row[2].split('@',1)[0].replace("'","")
            rxnode_key=rxnode+"_"+prodno
            print(rxnode_key)
            if rxnode_key in products_dict:
              product_associated_triples=products_dict[rxnode_key]
              for val in product_associated_triples:
                output_row=[]
                output_row.append(row[3])
                pred=val[1]
                if pred in identifier_source_list:
                  pred_notinwikidata_dict_fda_qualifier[pred]="PRX_ID_"+pred
                else:
                  pred_notinwikidata_dict_fda_qualifier[pred]="PRX_FDA_"+pred 
                
                pred=pred_notinwikidata_dict_fda_qualifier[pred]
                output_row.append(pred)
                output_row.append("\""+val[2]+"\"")
                output_rows_qnode_prxnode_fda_qualifier.append(output_row)
          elif row[1]=='PRX_FDA_submission_status_date':
            prodno=row[2].split('^',1)[1].replace("-","")
            rxnode_key=rxnode+"_"+prodno
            print(rxnode_key)
            if rxnode_key in submisions_dict:
              sub_associated_triples=submisions_dict[rxnode_key]
              for val in product_associated_triples:
                output_row=[]
                output_row.append(row[3])
                pred=val[1]
                if pred in identifier_source_list:
                  pred_notinwikidata_dict_fda_qualifier[pred]="PRX_ID_"+pred
                else:
                  pred_notinwikidata_dict_fda_qualifier[pred]="PRX_FDA_"+pred 
                
                pred=pred_notinwikidata_dict_fda_qualifier[pred]
                output_row.append(pred)
                output_row.append("\""+val[2]+"\"")
                output_rows_qnode_prxnode_fda_qualifier.append(output_row)
          else:
            prodno=row[2].split('@',1)[0].replace("'","")
            rxnode_key=rxnode+"_"+prodno
            print(rxnode_key)
            if rxnode_key in active_ingredients_dict:
              val=active_ingredients_dict[rxnode_key]
              
              output_row=[]
              output_row.append(row[3])
              pred='strength'
              if pred in identifier_source_list:
                pred_notinwikidata_dict_fda_qualifier[pred]="PRX_ID_"+pred
              else:
                pred_notinwikidata_dict_fda_qualifier[pred]="PRX_FDA_"+pred 
              
              pred=pred_notinwikidata_dict_fda_qualifier[pred]
              output_row.append(pred)
              output_row.append("\""+val+"\"")
              output_rows_qnode_prxnode_fda_qualifier.append(output_row)

print(output_rows_qnode_prxnode_fda_qualifier)


Header= ['node1', 'label', 'node2', 'id']
{'1656351_20160212': [['NDA207620', 'submission_status', 'AP'], ['NDA207620', 'submission_property_type', "{'id': '0'}"], ['NDA207620', 'submission_number', '1'], ['NDA207620', 'submission_class_code', 'MANUF (CMC)'], ['NDA207620', 'review_priority', 'PRIORITY'], ['NDA207620', 'submission_type', 'SUPPL'], ['NDA207620', 'submission_class_code_description', 'Manufacturing (CMC)']], '1656351_20160511': [['NDA207620', 'submission_status', 'AP'], ['NDA207620', 'submission_property_type', "{'id': '0'}"], ['NDA207620', 'submission_number', '2'], ['NDA207620', 'submission_class_code', 'MANUF (CMC)'], ['NDA207620', 'review_priority', 'PRIORITY'], ['NDA207620', 'submission_type', 'SUPPL'], ['NDA207620', 'submission_class_code_description', 'Manufacturing (CMC)']], '1656351_20171122': [['NDA207620', 'submission_status', 'AP'], ['NDA207620', 'submission_property_type', "{'id': '31'}"], ['NDA207620', 'application_docs', "{'date': '20171127', 'url': 'http://

In [67]:
write_tsv("../results/fda/kgtk_triples/qualifiers/qrxnode_prxnode_fda_qualifier.tsv", output_rows_qrxnode_prxnode_fda_qualifier)
write_tsv("../results/fda/kgtk_triples/qualifiers/qnode_prxnode_fda_qualifier.tsv", output_rows_qnode_prxnode_fda_qualifier)

In [68]:
output_rows_qrxnode_prxnode_rxnorm_qualifier=[]
output_rows_qrxnode_pnode_rxnorm_qualifier=[]

output_rows_qnode_prxnode_rxnorm_qualifier=[]
output_rows_qnode_pnode_rxnorm_qualifier=[]

def get_key(val, input_dict):
    for key, value in input_dict.items():
         if val == value:
             return key

with open('../results/rxnorm/kgtk_triples_id/nodes/qrxnode_prxnode_rxnorm_id.tsv', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter='\t')
    
    header_list = next(reader)
    print("Header=", header_list)
    
    #umlscui_dict={}
    print(rxcui_identifier_qual_dict)
   
    for row in reader:
      if row[1] in ("PRX_ID_NDDF", "PRX_ID_MMSL", "PRX_ID_VANDF", "PRX_ID_MMSX", "PRX_ID_MTHSPL", "PRX_ID_GS"):
        print(row[0], row[1])
        rxnode=row[0].split('QRX',1)[1]
        identifier=row[1].split('PRX_ID_',1)[1]
        rxnode_key=rxnode+"_"+identifier
        if rxnode_key in rxcui_identifier_qual_dict:
          alias_list=rxcui_identifier_qual_dict[rxnode_key]
          for val in alias_list:
            output_row=[]
            output_row.append(row[3])
            output_row.append('alias')
            output_row.append(val)
            output_rows_qrxnode_prxnode_rxnorm_qualifier.append(output_row)

print(output_rows_qrxnode_prxnode_rxnorm_qualifier)   


with open('../results/rxnorm/kgtk_triples_id/nodes/qrxnode_pnode_rxnorm_id.tsv', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter='\t')
    
    header_list = next(reader)
    print("Header=", header_list)
    
    #umlscui_dict={}
    #print(rxcui_identifier_qual_dict)
   
    for row in reader:
      if row[1] in ("P5806", "P486", "P715", "P267"):
        print(row[0], row[1])
        
        rxnode=row[0].split('QRX',1)[1]
        identifier=get_key(row[1], pred_wikidata_dict)
        rxnode_key=rxnode+"_"+identifier
        print(rxnode_key)
        if rxnode_key in rxcui_identifier_qual_dict:
          alias_list=rxcui_identifier_qual_dict[rxnode_key]
          for val in alias_list:
            output_row=[]
            output_row.append(row[3])
            output_row.append('alias')
            output_row.append(val)
            output_rows_qrxnode_pnode_rxnorm_qualifier.append(output_row)
            

print(output_rows_qrxnode_pnode_rxnorm_qualifier)   

with open('../results/rxnorm/kgtk_triples_id/nodes/qnode_prxnode_rxnorm_id.tsv', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter='\t')
    
    header_list = next(reader)
    print("Header=", header_list)
    
    #umlscui_dict={}
    print(rxcui_identifier_qual_dict)
   
    for row in reader:
      if row[1] in ("PRX_ID_NDDF", "PRX_ID_MMSL", "PRX_ID_VANDF", "PRX_ID_MMSX", "PRX_ID_MTHSPL", "PRX_ID_GS"):
        print(row[0], row[1])
        rxnode=get_key(row[0],qnode_dict_inwiki)
        identifier=row[1].split('PRX_ID_',1)[1]
        rxnode_key=rxnode+"_"+identifier
        if rxnode_key in rxcui_identifier_qual_dict:
          alias_list=rxcui_identifier_qual_dict[rxnode_key]
          for val in alias_list:
            output_row=[]
            output_row.append(row[3])
            output_row.append('alias')
            output_row.append(val)
            output_rows_qnode_prxnode_rxnorm_qualifier.append(output_row)

print(output_rows_qnode_prxnode_rxnorm_qualifier) 


with open('../results/rxnorm/kgtk_triples_id/nodes/qnode_pnode_rxnorm_id.tsv', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter='\t')
    
    header_list = next(reader)
    print("Header=", header_list)
    
    #umlscui_dict={}
    #print(rxcui_identifier_qual_dict)
   
    for row in reader:
      if row[1] in ("P5806", "P486", "P715", "P267"):
        print(row[0], row[1])
        
        rxnode=get_key(row[0],qnode_dict_inwiki)
        identifier=get_key(row[1], pred_wikidata_dict)
        rxnode_key=rxnode+"_"+identifier
        print(rxnode_key)
        if rxnode_key in rxcui_identifier_qual_dict:
          alias_list=rxcui_identifier_qual_dict[rxnode_key]
          for val in alias_list:
            output_row=[]
            output_row.append(row[3])
            output_row.append('alias')
            output_row.append(val)
            output_rows_qnode_pnode_rxnorm_qualifier.append(output_row)
            

print(output_rows_qnode_pnode_rxnorm_qualifier)

Header= ['node1', 'label', 'node2', 'id']
{'69749_USP': ['Valsartan'], '69749_GS': ['Valsartan'], '69749_SNOMEDCT_US': ['Valsartan-containing product', 'Valsartan', 'Valsartan (substance)', 'Product containing valsartan (medicinal product)'], '69749_VANDF': ['VALSARTAN'], '69749_MTHSPL': ['VALSARTAN', 'Valsartan'], '69749_NDDF': ['valsartan'], '69749_ATC': ['valsartan'], '69749_MMSL': ['valsartan', 'valsartan', 'Valsartan'], '69749_MSH': ['Valsartan', "L-Valine, N-(1-oxopentyl)-N-((2'-(1H-tetrazol-5-yl)(1,1'-biphenyl)-4-yl)methyl)-", "N-valeryl-N-((2'-(1H-tetrazol-5-yl)biphenyl-4-yl)methyl)valine"], '69749_DRUGBANK': ['N-(P-(O-1H-Tetrazol-5-ylphenyl)benzyl)-N-valeryl-L-valine', 'Valsartan', "N-pentanoyl-N-{[2'-(1H-tetrazol-5-yl)biphenyl-4-yl]methyl}-L-valine", "(S)-N-Valeryl-N-{[2'-(1H-tetrazol-5-yl)biphenyl-4-yl]-methyl}-valine"], '1151133_NDDF': ['PILL'], '1656328_GS': ['Sacubitril'], '1656328_DRUGBANK': ['Sacubitrilo', 'Sacubitril'], '1656328_MTHSPL': ['SACUBITRIL'], '1656328_MMSL':

In [69]:
write_tsv("../results/rxnorm/kgtk_triples/qualifiers/qrxnode_prxnode_rxnorm_qualifier.tsv", output_rows_qrxnode_prxnode_rxnorm_qualifier)
write_tsv("../results/rxnorm/kgtk_triples/qualifiers/qnode_prxnode_rxnorm_qualifier.tsv", output_rows_qnode_prxnode_rxnorm_qualifier)

write_tsv("../results/rxnorm/kgtk_triples/qualifiers/qrxnode_pnode_rxnorm_qualifier.tsv", output_rows_qrxnode_pnode_rxnorm_qualifier)
write_tsv("../results/rxnorm/kgtk_triples/qualifiers/qnode_pnode_rxnorm_qualifier.tsv", output_rows_qnode_pnode_rxnorm_qualifier)

In [70]:
output_rows_prxnode_fda_qualifier=[]
output_rows_prxnode_edges_fda_qualifier=[]
output_rows_prxnode_datatype_fda_qualifier=[]
for x in pred_notinwikidata_dict_fda_qualifier.keys():
    
    print(x)
    
    output_row=[]
    pred_value=pred_notinwikidata_dict_fda_qualifier[x]
    output_row.append(pred_value)
    output_row.append("label")
    output_row.append("\'"+x+"\'"+"@en")
    output_rows_prxnode_fda_qualifier.append(output_row)
    output_rows_prxnode_edges_fda_qualifier.append(output_row)
    
    output_row=[]
    output_row.append(pred_value)
    output_row.append("description")
    output_row.append("\'"+x+"\'"+"@en")
    output_rows_prxnode_fda_qualifier.append(output_row)
    output_rows_prxnode_edges_fda_qualifier.append(output_row)
    
    output_row=[]
    output_row.append(pred_value)
    output_row.append("data_type")
    if ("PRX_ID_" in pred_value):
        output_row.append("\""+"external-identifier"+"\"")
    elif ("PRX_FDA_" in pred_value):
        output_row.append("\""+"string"+"\"")
    #output_row.append("\"String\"")
    output_rows_prxnode_fda_qualifier.append(output_row)
    output_rows_prxnode_datatype_fda_qualifier.append(output_row)

print(output_rows_prxnode_fda_qualifier)

strength
reference_drug
route
marketing_status
brand_name
active_ingredients
reference_standard
dosage_form
[['PRX_FDA_strength', 'label', "'strength'@en"], ['PRX_FDA_strength', 'description', "'strength'@en"], ['PRX_FDA_strength', 'data_type', '"string"'], ['PRX_FDA_reference_drug', 'label', "'reference_drug'@en"], ['PRX_FDA_reference_drug', 'description', "'reference_drug'@en"], ['PRX_FDA_reference_drug', 'data_type', '"string"'], ['PRX_FDA_route', 'label', "'route'@en"], ['PRX_FDA_route', 'description', "'route'@en"], ['PRX_FDA_route', 'data_type', '"string"'], ['PRX_FDA_marketing_status', 'label', "'marketing_status'@en"], ['PRX_FDA_marketing_status', 'description', "'marketing_status'@en"], ['PRX_FDA_marketing_status', 'data_type', '"string"'], ['PRX_FDA_brand_name', 'label', "'brand_name'@en"], ['PRX_FDA_brand_name', 'description', "'brand_name'@en"], ['PRX_FDA_brand_name', 'data_type', '"string"'], ['PRX_FDA_active_ingredients', 'label', "'active_ingredients'@en"], ['PRX_FDA_act

In [71]:
write_tsv("../results/fda/kgtk_triples/properties/prxnode_fda_qualifier.tsv", output_rows_prxnode_fda_qualifier)
write_tsv("../results/fda/kgtk_triples/properties/prxnode_edges_fda_qualifier.tsv", output_rows_prxnode_edges_fda_qualifier)
write_tsv("../results/fda/kgtk_triples/properties/prxnode_datatype_fda_qualifier.tsv", output_rows_prxnode_datatype_fda_qualifier)

In [72]:
!kgtk compact -i ../results/rxnorm/kgtk_triples/qualifiers/qrxnode_pnode_rxnorm_qualifier.tsv -o ../results/rxnorm/kgtk_triples_compact/qualifiers/qrxnode_pnode_rxnorm_qualifier_compact.tsv
!kgtk compact -i ../results/rxnorm/kgtk_triples/qualifiers/qrxnode_prxnode_rxnorm_qualifier.tsv -o ../results/rxnorm/kgtk_triples_compact/qualifiers/qrxnode_prxnode_rxnorm_qualifier_compact.tsv
!kgtk compact -i ../results/rxnorm/kgtk_triples/qualifiers/qnode_pnode_rxnorm_qualifier.tsv -o ../results/rxnorm/kgtk_triples_compact/qualifiers/qnode_pnode_rxnorm_qualifier_compact.tsv
!kgtk compact -i ../results/rxnorm/kgtk_triples/qualifiers/qnode_prxnode_rxnorm_qualifier.tsv -o ../results/rxnorm/kgtk_triples_compact/qualifiers/qnode_prxnode_rxnorm_qualifier_compact.tsv

In [73]:
!kgtk compact -i ../results/fda/kgtk_triples/qualifiers/qrxnode_prxnode_fda_qualifier.tsv -o ../results/fda/kgtk_triples_compact/qualifiers/qrxnode_prxnode_fda_qualifier_compact.tsv
!kgtk compact -i ../results/fda/kgtk_triples/qualifiers/qnode_prxnode_fda_qualifier.tsv -o ../results/fda/kgtk_triples_compact/qualifiers/qnode_prxnode_fda_qualifier_compact.tsv

In [74]:
!kgtk compact -i ../results/fda/kgtk_triples/properties/prxnode_edges_fda_qualifier.tsv -o ../results/fda/kgtk_triples_compact/properties/prxnode_edges_fda_qualifier_compact.tsv

In [75]:
!kgtk add-id -i ../results/rxnorm/kgtk_triples_compact/qualifiers/qrxnode_pnode_rxnorm_qualifier_compact.tsv -o ../results/rxnorm/kgtk_triples_id/qualifiers/qrxnode_pnode_rxnorm_qualifier_id.tsv --id-style node1-label-node2-num
!kgtk add-id -i ../results/rxnorm/kgtk_triples_compact/qualifiers/qrxnode_prxnode_rxnorm_qualifier_compact.tsv -o ../results/rxnorm/kgtk_triples_id/qualifiers/qrxnode_prxnode_rxnorm_qualifier_id.tsv --id-style node1-label-node2-num
!kgtk add-id -i ../results/rxnorm/kgtk_triples_compact/qualifiers/qnode_pnode_rxnorm_qualifier_compact.tsv -o ../results/rxnorm/kgtk_triples_id/qualifiers/qnode_pnode_rxnorm_qualifier_id.tsv --id-style node1-label-node2-num
!kgtk add-id -i ../results/rxnorm/kgtk_triples_compact/qualifiers/qnode_prxnode_rxnorm_qualifier_compact.tsv -o ../results/rxnorm/kgtk_triples_id/qualifiers/qnode_prxnode_rxnorm_qualifier_id.tsv --id-style node1-label-node2-num

In [76]:
!kgtk add-id -i ../results/fda/kgtk_triples_compact/qualifiers/qrxnode_prxnode_fda_qualifier_compact.tsv -o ../results/fda/kgtk_triples_id/qualifiers/qrxnode_prxnode_fda_qualifier_id.tsv --id-style node1-label-node2-num
!kgtk add-id -i ../results/fda/kgtk_triples_compact/qualifiers/qnode_prxnode_fda_qualifier_compact.tsv -o ../results/fda/kgtk_triples_id/qualifiers/qnode_prxnode_fda_qualifier_id.tsv --id-style node1-label-node2-num

In [77]:
!kgtk add-id -i ../results/fda/kgtk_triples_compact/properties/prxnode_edges_fda_qualifier_compact.tsv -o ../results/fda/kgtk_triples_id/properties/prxnode_edges_fda_qualifier_id.tsv --id-style node1-label-node2-num

In [78]:
!kgtk validate -i ../results/rxnorm/kgtk_triples_id/qualifiers/qrxnode_pnode_rxnorm_qualifier_id.tsv -v
!kgtk validate -i ../results/rxnorm/kgtk_triples_id/qualifiers/qrxnode_prxnode_rxnorm_qualifier_id.tsv -v
!kgtk validate -i ../results/rxnorm/kgtk_triples_id/qualifiers/qnode_pnode_rxnorm_qualifier_id.tsv -v
!kgtk validate -i ../results/rxnorm/kgtk_triples_id/qualifiers/qnode_prxnode_rxnorm_qualifier_id.tsv -v


Validating '../results/rxnorm/kgtk_triples_id/qualifiers/qrxnode_pnode_rxnorm_qualifier_id.tsv'
KgtkReader: File_path.suffix: .tsv
KgtkReader: reading file ../results/rxnorm/kgtk_triples_id/qualifiers/qrxnode_pnode_rxnorm_qualifier_id.tsv
header: node1	label	node2	id
node1 column found, this is a KGTK edge file
KgtkReader: Special columns: node1=0 label=1 node2=2 id=3
KgtkReader: Reading an edge file.
Validated 2 data lines

Validating '../results/rxnorm/kgtk_triples_id/qualifiers/qrxnode_prxnode_rxnorm_qualifier_id.tsv'
KgtkReader: File_path.suffix: .tsv
KgtkReader: reading file ../results/rxnorm/kgtk_triples_id/qualifiers/qrxnode_prxnode_rxnorm_qualifier_id.tsv
header: node1	label	node2	id
node1 column found, this is a KGTK edge file
KgtkReader: Special columns: node1=0 label=1 node2=2 id=3
KgtkReader: Reading an edge file.
Validated 32 data lines

Validating '../results/rxnorm/kgtk_triples_id/qualifiers/qnode_pnode_rxnorm_qualifier_id.tsv'
KgtkReader: File_path.suffix: .tsv
KgtkRea

In [79]:
!kgtk validate -i ../results/fda/kgtk_triples_id/qualifiers/qrxnode_prxnode_fda_qualifier_id.tsv -v
!kgtk validate -i ../results/fda/kgtk_triples_id/qualifiers/qnode_prxnode_fda_qualifier_id.tsv -v


Validating '../results/fda/kgtk_triples_id/qualifiers/qrxnode_prxnode_fda_qualifier_id.tsv'
KgtkReader: File_path.suffix: .tsv
KgtkReader: reading file ../results/fda/kgtk_triples_id/qualifiers/qrxnode_prxnode_fda_qualifier_id.tsv
header: node1	label	node2	id
node1 column found, this is a KGTK edge file
KgtkReader: Special columns: node1=0 label=1 node2=2 id=3
KgtkReader: Reading an edge file.
Validated 69 data lines

Validating '../results/fda/kgtk_triples_id/qualifiers/qnode_prxnode_fda_qualifier_id.tsv'
KgtkReader: File_path.suffix: .tsv
KgtkReader: reading file ../results/fda/kgtk_triples_id/qualifiers/qnode_prxnode_fda_qualifier_id.tsv
header: node1	label	node2	id
node1 column found, this is a KGTK edge file
KgtkReader: Special columns: node1=0 label=1 node2=2 id=3
KgtkReader: Reading an edge file.
Validated 0 data lines


In [80]:
!kgtk validate -i ../results/fda/kgtk_triples_id/properties/prxnode_edges_fda_qualifier_id.tsv -v


Validating '../results/fda/kgtk_triples_id/properties/prxnode_edges_fda_qualifier_id.tsv'
KgtkReader: File_path.suffix: .tsv
KgtkReader: reading file ../results/fda/kgtk_triples_id/properties/prxnode_edges_fda_qualifier_id.tsv
header: node1	label	node2	id
node1 column found, this is a KGTK edge file
KgtkReader: Special columns: node1=0 label=1 node2=2 id=3
KgtkReader: Reading an edge file.
Validated 16 data lines


In [81]:
!kgtk cat -i ../results/rxnorm/kgtk_triples_id/nodes/qrxnode_pnode_rxnorm_id.tsv ../results/rxnorm/kgtk_triples_id/nodes/qrxnode_prxnode_rxnorm_id.tsv ../results/rxnorm/kgtk_triples_id/nodes/qnode_pnode_rxnorm_id.tsv ../results/rxnorm/kgtk_triples_id/nodes/qnode_prxnode_rxnorm_id.tsv -o ../results/rxnorm/kgtk_merged/nodes_edges_merged_rxnorm.tsv

In [82]:
!kgtk cat -i ../results/rxnorm/kgtk_triples_id/qualifiers/qrxnode_pnode_rxnorm_qualifier_id.tsv ../results/rxnorm/kgtk_triples_id/qualifiers/qrxnode_prxnode_rxnorm_qualifier_id.tsv ../results/rxnorm/kgtk_triples_id/qualifiers/qnode_pnode_rxnorm_qualifier_id.tsv ../results/rxnorm/kgtk_triples_id/qualifiers/qnode_prxnode_rxnorm_qualifier_id.tsv -o ../results/rxnorm/kgtk_merged/qualifiers_edges_merged_rxnorm.tsv

In [83]:
!kgtk cat -i ../results/rxnorm/kgtk_triples_id/properties/prxnode_edges_rxnorm_id.tsv -o ../results/rxnorm/kgtk_merged/properties_edges_merged_rxnorm.tsv
!kgtk cat -i ../results/rxnorm/kgtk_triples/properties/prxnode_datatype_rxnorm.tsv -o ../results/rxnorm/kgtk_merged/properties_datatype_merged_rxnorm.tsv

In [84]:
!kgtk cat -i ../results/fda/kgtk_triples_id/nodes/qrxnode_pnode_fda_id.tsv ../results/fda/kgtk_triples_id/nodes/qrxnode_prxnode_fda_id.tsv ../results/fda/kgtk_triples_id/nodes/qnode_pnode_fda_id.tsv ../results/fda/kgtk_triples_id/nodes/qnode_prxnode_fda_id.tsv -o ../results/fda/kgtk_merged/nodes_edges_merged_fda.tsv

In [85]:
!kgtk cat -i ../results/fda/kgtk_triples_id/qualifiers/qrxnode_prxnode_fda_qualifier_id.tsv ../results/fda/kgtk_triples_id/qualifiers/qnode_prxnode_fda_qualifier_id.tsv -o ../results/fda/kgtk_merged/qualifiers_edges_merged_fda.tsv

In [86]:
!kgtk cat -i ../results/fda/kgtk_triples_id/properties/prxnode_edges_fda_id.tsv ../results/fda/kgtk_triples_id/properties/prxnode_edges_fda_qualifier_id.tsv -o ../results/fda/kgtk_merged/properties_edges_merged_fda.tsv
!kgtk cat -i ../results/fda/kgtk_triples/properties/prxnode_datatype_fda.tsv ../results/fda/kgtk_triples/properties/prxnode_datatype_fda_qualifier.tsv -o ../results/fda/kgtk_merged/properties_datatype_merged_fda.tsv

In [87]:
!kgtk cat -i ../results/rxnorm/kgtk_merged/nodes_edges_merged_rxnorm.tsv ../results/rxnorm/kgtk_merged/properties_edges_merged_rxnorm.tsv ../results/rxnorm/kgtk_merged/qualifiers_edges_merged_rxnorm.tsv -o ../results/merged_ingestion/intermediate/nodes_properties_qualifiers_alledges_rxnorm.tsv

In [88]:
!kgtk cat -i ../results/fda/kgtk_merged/nodes_edges_merged_fda.tsv ../results/fda/kgtk_merged/properties_edges_merged_fda.tsv ../results/fda/kgtk_merged/qualifiers_edges_merged_fda.tsv -o ../results/merged_ingestion/intermediate/nodes_properties_qualifiers_alledges_fda.tsv

In [89]:
!kgtk cat -i ../results/rxnorm/kgtk_merged/properties_datatype_merged_rxnorm.tsv ../results/fda/kgtk_merged/properties_datatype_merged_fda.tsv -o ../results/merged_ingestion/final/properties_datatype_merged.tsv

In [90]:
!kgtk cat -i ../results/merged_ingestion/intermediate/nodes_properties_qualifiers_alledges_rxnorm.tsv ../results/merged_ingestion/intermediate/nodes_properties_qualifiers_alledges_fda.tsv -o ../results/merged_ingestion/final/nodes_properties_qualifiers_alledges_merged.tsv